# TP6 : Maximum de vraisemblance via l'algorithme de montée de gradient

In [ ]:
# Chargement des bibliothèques
from matplotlib import pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs

print('Bibliothèques chargées')

## TD : Problème A

### Partie 1 : résolution directe

On considère les données $X_j=(X_{j1},X_{j2})\mapsto Y_j$, pour $j\in\{1,2,3,4\}$, avec :

$X_1=(2,0)$, $X_2=(3,2)$, $X_3=(0,3)$. $X_3=(3,4)$, et

$Y_1=0$, $Y_2=0$, $Y_3=1$, $Y_4=1$.

La cellule suivante permet de représenter les données - recopier ou représenter vous-même ce graphique sur papier.

In [ ]:
# Définition des données

J=4 # Nb de données
X1=[2,3,0,3]
X2=[0,2,3,4]
Y=[0,0,1,1]

#Représentation graphique des données

plt.axis([-1,5,-1,5]) #xmin,xmax,ymin,ymax
for i in range(J):
    if Y[i]==0:
        color='blue'
    else:
        color='red'
    plt.plot([X1[i]],[X2[i]],'o',c=color)
plt.show()

Dans la suite, on va chercher des coefficients $b$ et $W=\begin{pmatrix}w_1\\w_2\end{pmatrix}$ maximisant la vraisemblance de la loi de probabilité définie par $$\Pr(Y=1|X)=\sigma(b+w_1x_1+w_2x_2).$$
où $\sigma$ est la fonction sigmoïde, définie pour tout $x$ dans $R$ par $$\sigma(x)=\frac{1}{1+e^{-x}}.$$

La vraisemblance de cette loi est donnée par 
$$L(b,W)=\prod\limits_{j:Y_j=1}\sigma(b+w_1x_{j1}+w_2x_{j2})\times\prod\limits_{j:Y_j=0}(1-\sigma(b+w_1x_{j1}+w_2x_{j2}))$$

On rappelle (vu en cours) que $\sigma(x)\simeq 1$ dès que $x>>0$ ($x$ très supérieur à $0$), et $\sigma(x)\simeq 0$ dès que $x<<0$ ($x$ très inférieur à $0$).

La vraisemblance est donc grande si les coefficients $b$, $w_1$ et $w_2$ vérifient pour tout $j\in\{1,2,3,4\}$ :

$$(C_j):\left\{\begin{array}{l}b+w_1x_{j1}+w_2x_{j2}>>0\text{ si }Y_j=1\\b+w_1x_{j1}+w_2x_{j2}<<0\text{ si }Y_j=0\end{array}\right.$$


**Exercice**

1. Ecrire les 4 conditions $(C_j)$ que doivent satisfaire $b$, $w_1$ et $w_2$ :
  - Condition (C1) issue de la donnée X1,Y1 : ...
  - Condition (C2) issue de la donnée X2,Y2 : ...
  - Condition (C3) issue de la donnée X3,Y3 : ...
  - Condition (C4) issue de la donnée X4,Y4 : ...

2. Proposer une valeur pour $b$, $w_1$, $w_2$ satisfaisant les 4 conditions précédentes.

Réponse : b=.... , w1=.... , w2=....

*Indication* : pour vous aider, tracer sur papier une droite séparant les données $Y_j$0$ des données $Y_j=1$. Cette droite a pour équation b+w_1x_1+w_2x_2=0$. Vous pouvez déterminer $b$, $w_1$ et $w_2$ à partir des conditions :
- le gradient $(w_1,w_2)$ de cette droite est orienté vers les $Y=1$ ;
- les coordonnées d'un point $Z$ (à choisir) de la droite satisfont l'équation de la droite : $b+w_1x_1(Z)+w_2x_2(Z)=0$ ;
- les coordonnées d'un point $Z$ (à choisir) situé dans la région associée aux $Y_j=0$ doit satsifaire : $b+w_1x_1(Z)+w_2x_2(Z)<<0$ ;
- les coordonnées d'un point $Z$ (à choisir) situé dans la région associée aux $Y_j=1$ doit satsifaire : $b+w_1x_1(Z)+w_2x_2(Z)>>0$ ;


### Aide à la compréhension : visualisation de la solution

In [ ]:
# Définition de la fonction sigma
def sigma(x):
    return 1/(1+np.exp(-x))

print('Fonction sigma enregistrée')

La cellule ci-dessous permet de compléter le graphique généré avec les courbes de niveaux de la fonctionn $(x_1,x_2)\mapsto\sigma(b+w_1x_1+w_2x_2)$ trouvée dans l'exercice précédent.

Noter que la fonction associe bien une valeur proche de 0 d'un côté de la droite et une valeur proche de 1 de l'autre côté de la droite, comme l'exigent les données recueillies.

In [ ]:
# Courbes de niveau de la fonction sigma(b+w1x1+w2x2)

# Calcul du tableau de valeurs
x1list=np.linspace(-1,5,50) # liste des x1
x2list=np.linspace(-1,5,50) # liste des x2
# compléter la ligne suivante avec les bonnes valeurs de b, w1, w2
sigmalist=[[sigma(1+2*x1+3*x2) for x1 in x1list] for x2 in x2list] # liste des valeurs de la fonction

# Dessin des courbes de niveau
plt.axis([-1,5,-1,5])
plt.contour(x1list,x2list,sigmalist,100) # dessine 100 courbes de niveau de sigma(b+w1x1+w2x2)
plt.colorbar()

# Affichage des données initiales
for i in range(len(X1)):
    if Y[i]==0:
        color='blue'
    else:
        color='red'
    plt.plot([X1[i]],[X2[i]],'o',c=color)
plt.show()

### Partie 2 : Résolution via l'algorithme de montée de gradient

Dans la suite, on va déterminer une fonction $(x_1,x_2)\mapsto \sigma(b+w_1x_1+w_2x_2)$ séparant les données à l'aide d'un algorithme de *montée* de gradient sur la fonction de *log-vraisemblance*, qu'il s'agit de *maximiser*.

On rappelle que la log vraisemblance est donnée par :
$$\ln(L(b,W))=\sum\limits_{j:Y_j=1}\ln(\sigma(b+w_1x_{j1}+w_2x_{j2}))+\sum\limits_{j:Y_j=0}\ln((1-\sigma(b+w_1x_{j1}+w_2x_{j2})))$$

Pour effectuer la montée de gradient, on a besoin des dérivées de la log vraisemblance, données par (cf cours) :
- $\frac{\partial \ln(L(b,W))}{\partial b}=\sum\limits_j\left(Y_j-\sigma(b+w_1x_{j1}+w_2x_{j_2})\right)$
- $\frac{\partial \ln(L(b,W))}{\partial w_1}=\sum\limits_j\left(Y_j-\sigma(b+w_1x_{j1}+w_2x_{j_2})\right)x_{j1}$
- $\frac{\partial \ln(L(b,W)}{\partial w_2}=\sum\limits_j\left(Y_j-\sigma(b+w_1x_{j1}+w_2x_{j_2})\right)x_{j2}$


**Exercice**

On rappelle que $\ln'(x)=1/x$ et $\sigma'(x)=\sigma(x)\times (1-\sigma(x))$.

1. En déduire la dérivée de $\ln(\sigma(u(x))$ en utilisant la formule de dérivée d'une fonction composée $(U\circ V)'(x)=U'(V(x))\times V'(x)$.

2. En déduire l'expression de $\frac{\partial \ln(L(b,W)}{\partial w_2}$ comme donnée dans la cellule précédente.

### Montée de gradient

Le principe de la montée de gradient est de calculer, à partir d'une initialisation à $b[0]$, $w_1[0]$ et $w_2[0]$, la sequence $b[i]$, $w_1[i]$, $w_2[i]$, avec $i=1,2,\ldots$, telle que, en chaque $i$:

- $b[i+1]=b[i]+\tau\times\frac{\partial \ln(L(b,W))}{\partial b}$
- $w_1[i+1]=w_1[i]+\tau\times\frac{\partial \ln(L(b,W))}{\partial w_1}$
- $w_2[i+1]=w_2[i]+\tau\times\frac{\partial \ln(L(b,W))}{\partial w_2}$



**Exercice**

On initialise le gradient à $b[0]=-1$, $w_1[0]=0$ et $w_2[0]=1$.
On choisit un pas $\tau=0.1$.
1. Quelle est l'équation de la droite sur laquelle est initialisé l'algorithme ? Représenter cette droite sur votre graphique.
2. On donne $[\text{grad}\ln(L)](-1,0,1)=(-0.76,-2.59,-0.91)$.
   - Que valent $b[1]$, $w_1[1]$ et $w_2[1]$ ?
   - En déduire l'équation de la droite obtenue après une itération de l'algorithme. Représenter cette droite sur votre graphique.
   - L'algorithme vous semble-t-il "aller dans le bon sens" ?

La cellule ci-dessous définit les fonctions :
- `gradlnL(b,w1,w2)`, qui renvoit le gradient de la log-vraisemblance calculée en (b,w1,w2)
- `montee(b,w1,w2,tau,tolerance,nbiterationsmax)`, qui effectue l'algorithme de montee de gradient et renvoit la liste `[b[i],w1[i],w2[i]]` des coefficients atteints par l'algorithme.


In [ ]:
# Fonction renvoyant le gradient de la log vraisemblance associée à b,w1,w2
def gradlnL(b,w1,w2):
    return [np.sum([Y[j]-sigma(b+w1*X1[j]+w2*X1[j]) for j in range(J)]),
            np.sum([(Y[j]-sigma(b+w1*X1[j]+w2*X2[j]))*X1[j] for j in range(J)]),
            np.sum([(Y[j]-sigma(b+w1*X1[j]+w2*X2[j]))*X2[j] for j in range(J)])
           ] 

print('Fonction gradlnL enregistrée')

# Algorithme de montee de gradient initialisé
def montee(b,w1,w2,tau,tolerance,NbIterationsMax):
    L=[]
    for i in range(NbIterationsMax):
        try: # traitement des erreurs si l'algorithme diverge
            if (b==float('inf')) or (w1==float("inf")) or (w2==float("inf")):
                raise(OverflowError)
            g = gradlnL(b,w1,w2) # calcul du gradient
            if g[0]**2+g[1]**2+g[2]**2<tolerance: # si le gradient est suffisamment petit
                print('L\'algorithme a convergé en',i,'itérations. \nSolution atteinte :\n b=',b,'\n w1=',w1,'\n w2=',w2,'\nGradient :',g,'\nNorme du gradient:',g[0]**2+g[1]**2+g[2]**2)
                L.append([b,w1,w2])
                return L
            L.append([b,w1,w2]) # On ajoute les valeurs obtenues dans une liste pour visualiser la progression de l'algorithme ultérieurement
            # Mise à jour des valeurs de b,w1 et w2
            b=b+tau*g[0]
            w1=w1+tau*g[1]
            w2=w2+tau*g[2]
        except OverflowError as err: # traitement de l'erreur "overflow"
            print('L\'algorithme a divergé.\n Solution atteinte :\n b=',b,'\n w1=',w1,'\n w2=',w2,'\nGradient :',g,'\nNorme du gradient:',g[0]**2+g[1]**2+g[2]**2)
            return L
    print('L\'algorithme n\'a pas convergé.\n Solution atteinte :\n b=',b,'\n w1=',w1,'\n w2=',w2,'\nGradient :',g,'\nNorme du gradient:',g[0]**2+g[1]**2+g[2]**2)
    return L

print('Fonction montee enregistrée')

### Aide à la compréhension

La cellule ci-dessous permet d'effectuer l'algorithme, puis de représenter les droites d'équation `b[i]+w1[i]*x1+w2[i]*x2=0` atteintes par l'algorithme de montée de gradient.

On dit que l'algorithme converge si le gradient de la log-vraisemblance atteint est suffisamment petit (inférieur à `tolerance`), avant que le nombre d'itérations de l'algorithme ne dépasse un certain seuil (`NbIterationsMax`).

Tester l'algorithme en l'initialisant à $b=-1$, $w_1=0$, $w_2=1$, avec un pas $\tau=0.05$, une tolérance de $0.01$ et un nombre d'itérations maximum de 500. 

Vous devriez obtenir une convergence en 241 itérations.

Vérifier sur le graphique que la droite obtenue sépare bien les données comme souhaité.

In [ ]:
# Calculs de la suite des [b[i],w1[i],w2[i]] via l'algorithme

# b=0, w1=0, w
ListeBW=montee(b=-1,w1=0,w2=1,tau=0.05,tolerance=0.01,NbIterationsMax=500)

# Représentation graphique des droites b[i]+w1[i]*x1+w2[i]*x2
for i in range(len(ListeBW)):
    W=ListeBW[i]
    b=W[0]
    w1=W[1]
    w2=W[2]
    # alpha est un paramètre de transparence variant ici de 0.1 à 0.9 avec la valeur de i
    transparence=max(0,min(1,0.1+0.9*i/(len(ListeBW)-1)))
    if (w2!=0):
        plt.plot([-1,5],[(-w1*(-1)-b)/w2,(-w1*5-b)/w2],c="black",alpha=transparence)
    elif (w1!=0):
        plt.plot([(-w2*(-1)-b)/w1,(-w2*5-b)/w1],[-1,5],c="black",alpha=transparence)        

# Nuage de points

plt.axis([-1,5,-1,5])
for i in range(len(X1)):
    if Y[i]==0:
        color='blue'
    else:
        color='red'
    plt.plot([X1[i]],[X2[i]],'o',c=color)
plt.show()

**Exercice**

1. D'après le graphique ci-dessous, à partir de quelle itération $i$ les paramètres $b[i]$, $w_1[i]$ et $w_2[i]$ obtenus permettent-ils de séparer les données ?
2. Explorer l'algorithme en ne changeant que le pas afin de trouver:
   - un cas où l'algorithme converge en moins de 50 itérations ;
   - un cas où l'algorithme converge en environ 500 itérations.

## Application à l'analyse de données

Les cellules suivantes permettent de généraliser les calculs précédents avec un nuage de 40 données.

In [ ]:
# Création aléatoire des données

# Cette cellule n'est à exécuter qu'une fois, afin de générer des données aléatoires,
# et afin de les recopier dans la cellule suivante

# on génère 40 données (X1j,X2j,Yj), avec Yj=0 ou 1
# données centrées autour de 2 centres
X, Y = make_blobs(n_samples=40, centers=[[1,3],[3,1]], n_features=2, cluster_std=1,center_box=(-1,1))
print('Liste des données (à recopier-coller dans la cellule ci-dessous pour une nouvelle expérimentation)')
print('Liste des Xj1 : \n\n',np.array2string(X[:,0],separator=',',max_line_width=10000).replace('\n', ''),'\n')
print('Liste des Xj2 : \n\n',np.array2string(X[:,1],separator=',',max_line_width=10000).replace('\n', ''),'\n')
print('\nListe des Yj : \n\n',np.array2string(Y,separator=',',max_line_width=10000).replace('\n', ''),'\n')

In [ ]:
# Modifier en copiant collant avec les données générées aléatoirement ci-dessus
# attention à bien identifier les crochets fermant et ouvrant
J=40
X1=[-0.86715598, 0.16305324, 1.45065167, 1.02801687, 2.59938211,-0.30718283, 1.20717298, 2.87475429, 1.79229859, 3.21035441, 0.35900084, 2.17704943, 1.31621245, 3.80784333,-0.50768321, 0.21866706, 2.97810756,-0.01457319, 1.93684753, 2.75633546, 1.21287286, 3.66884845, 3.42487525, 3.13435741, 4.89330045, 0.63037281, 0.32033049, 2.16387134, 0.74116708, 0.88673248, 2.99927601, 4.27286885, 0.24338331, 3.3682433 , 1.22226934, 4.10066762, 4.19736891, 1.62077801, 2.50169767, 2.46594702] 
X2=[ 1.6618172 , 3.85006485, 0.58783353, 4.26723368, 1.79095464, 3.6170258 , 3.11519913, 1.01814067, 5.22233833,-0.81649863, 1.85375971, 2.32170593, 3.26517827, 2.3542536 , 2.00036918, 3.22515041, 1.2575142 , 2.41431842, 0.41460761, 1.06821644, 2.34770238, 1.19239495, 1.56641641, 0.85711164,-0.54328645, 0.66010703, 2.35544006,-0.42526025, 3.45518438, 4.52836897,-0.22380754, 2.05485337, 3.31742022, 3.21233414, 1.52003253,-0.03167243, 0.42861576, 1.81032212, 3.08864858, 1.00224623] 
Y=[0,0,1,0,1,0,0,1,0,1,0,0,1,1,0,0,1,0,1,1,0,1,1,1,1,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1] 

In [ ]:
# Représentation graphique du nuage de points

plt.axis([-1,5,-1,5]) # xmin,xmax,ymin,ymax

for i in range(J):
    if Y[i]==0:
        color='blue'
    else:
        color='red'
    plt.plot([X1[i]],[X2[i]],'o',c=color)
plt.show()

**Exercice**

1. Les données effectivement observées sont-elles linéairement séparables ?
2. Une régression logistique vous semble-t-elle toutefois pertinente pour modéliser la génération de ces données ?
1. Déterminer une droite séparant bien les données sans utiliser l'algorithme de montée de gradient.

3. Utiliser la cellule suivante pour représenter les étapes de l'algorithme ainsi que le nuage de points.
4. Quels sont les nombres de faux positifs et de faux négatif pour les données observées ?

Pour une loi $(x1,x2)\mapsto \sigma(b+w_1x_1+w_2x_2)$ obtenue,
- un *faux positif* est tel que $\sigma(b+w_1x_{j1}+w_2x_{j2})>0.5$ alors que $Y_j=0$ ;
- un *faux négatif* est tel que $\sigma(b+w_1x_{j1}+w_2x_{j2})<0.5$ alors que $Y_j=1$).

In [ ]:
# Calculs de la suite des [b[i],w1[i],w2[i]] via l'algorithme

# b=0, w1=0, w2=1
ListeBW=montee(b=-1,w1=0,w2=1,tau=0.05,tolerance=0.01,NbIterationsMax=500)

# Représentation graphique des droites b[i]+w1[i]*x1+w2[i]*x2
for i in range(len(ListeBW)):
    W=ListeBW[i]
    b=W[0]
    w1=W[1]
    w2=W[2]
    # alpha est un paramètre de transparence variant ici de 0.1 à 0.9 avec la valeur de i
    transparence=max(0,min(1,0.1+0.9*i/(len(ListeBW)-1)))
    if (w2!=0):
        plt.plot([-1,5],[(-w1*(-1)-b)/w2,(-w1*5-b)/w2],c="black",alpha=transparence)
    elif (w1!=0):
        plt.plot([(-w2*(-1)-b)/w1,(-w2*5-b)/w1],[-1,5],c="black",alpha=transparence)        

# Nuage de points

plt.axis([-1,5,-1,5])
for i in range(len(X1)):
    if Y[i]==0:
        color='blue'
    else:
        color='red'
    plt.plot([X1[i]],[X2[i]],'o',c=color)
plt.show()

In [ ]:
# Représentation graphique


**Réponses à compléter **

- Valeurs initiales : ...

- tolerance : ..., facteur $\tau$ : ...
- Convergence en ... itérations

- Droite obtenu : ...

- Nombre de faux positifs :

- Nombre de faux négatifs :